# Seq2Seq,Attention
## 参考资料
#### 课件
- [cs224d](http://cs224d.stanford.edu/lectures/CS224d-Lecture15.pdf)


#### 论文
- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
- [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025?context=cs)
- [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1406.1078)


#### PyTorch代码
- [seq2seq-tutorial](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb)
- [Tutorial from Ben Trevett](https://github.com/bentrevett/pytorch-seq2seq)
- [IBM seq2seq](https://github.com/IBM/pytorch-seq2seq)
- [OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py)


#### 更多关于Machine Translation
- [Beam Search](https://www.coursera.org/lecture/nlp-sequence-models/beam-search-4EtHZ)
- Pointer network 文本摘要
- Copy Mechanism 文本摘要
- Converage Loss 
- ConvSeq2Seq
- Transformer
- Tensor2Tensor

#### TODO
- 建议尝试对中文进行分词

#### NER
- https://github.com/allenai/allennlp/tree/master/allennlp


在这份notebook当中，复现的是Luong的attention模型

In [100]:
import os
import sys
import math
import numpy as np
import random
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F

import nltk
import jieba


读入中英文数据
- 英文我们使用nltk的word tokenizer来分词，并且使用小写字母
- 中文我们直接使用单个汉字作为基本单元

In [101]:
def load_data(in_file):
    ''' 将数据加载进来 '''
    cn = []
    en = []
    num_examples = 0
    with open(in_file, 'r',encoding="utf-8-sig") as f: #encoding="utf-8-sig" 处理中文，后边的 -sig 是为了去掉 ufeff
        for line in f:
            #print(line) #Anyone can do that.	任何人都可以做到。
            line = line.strip().split("\t") #分词后用逗号隔开
            #print(line) #['Anyone can do that.', '任何人都可以做到。']
            #en.append(["BOS"] + [c for c in line[0].lower().split()] + ["EOS"])
            en.append(["BOS"] + nltk.word_tokenize(line[0].lower()) + ["EOS"])
            #BOS:beginning of sequence EOS:end of
            #nltk.word_tolenize(): split English sentence
            
            # split Chinese sentence into characters
            cn.append(["BOS"] + [c for c in jieba.cut(line[1])] + ["EOS"])
           
    return en, cn

train_file = "nmt/en-cn/train.txt"
dev_file = "nmt/en-cn/dev.txt"
train_en, train_cn = load_data(train_file)
dev_en, dev_cn = load_data(dev_file)

In [102]:
train_en[0]

['BOS', 'anyone', 'can', 'do', 'that', '.', 'EOS']

### 构建单词表

In [103]:
UNK_IDX = 0
PAD_IDX = 1
def build_dict(sentences, max_words=50000):
    ''' 构建字典 '''
    word_count = Counter()
    for sentence in sentences:
        for s in sentence:
            word_count[s] += 1  #word_count这里应该是个字典
    ls = word_count.most_common(max_words) 
    #按每个单词数量排序前50000个,这个数字自己定的，不重复单词数没有50000
    print(len(ls)) #train_en：5491
    total_words = len(ls) + 2
    #加的2是留给"unk"和"pad"
    #ls = [('BOS', 14533), ('EOS', 14533), ('.', 12521), ('i', 4045), .......
    word_dict = {w[0]: index+2 for index, w in enumerate(ls)}
    #加的2是留给"unk"和"pad",转换成字典格式。
    word_dict["UNK"] = UNK_IDX
    word_dict["PAD"] = PAD_IDX
    return word_dict, total_words

en_dict, en_total_words = build_dict(train_en)
cn_dict, cn_total_words = build_dict(train_cn)
inv_en_dict = {v: k for k, v in en_dict.items()}
#en_dict.items()把字典转换成可迭代对象，取出键值，并调换键值的位置。
inv_cn_dict = {v: k for k, v in cn_dict.items()}

5491
11264


### 将预料数字化

In [104]:
def encode(en_sentences, cn_sentences, en_dict, cn_dict, sort_by_len=True):
    '''
        Encode the sequences. 
    '''
    length = len(en_sentences)
    #en_sentences=[['BOS', 'anyone', 'can', 'do', 'that', '.', 'EOS'],....
    
    out_en_sentences = [[en_dict.get(w, 0) for w in sent] for sent in en_sentences]
    #out_en_sentences=[[2, 328, 43, 14, 28, 4, 3], ....
    #.get(w, 0)，返回w对应的值，没有就为0.因题库比较小，这里所有的单词向量都有非零索引。
    
 
    out_cn_sentences = [[cn_dict.get(w, 0) for w in sent] for sent in cn_sentences]
    
    # sort sentences by english lengths
    def len_argsort(seq):
        
        '''
        根据句子长度对文本进行排序
        #sorted()排序,key参数可以自定义规则，按seq[x]的长度排序，seq[0]为第一句话长度
        '''
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))
      
       
    # 把中文和英文按照同样的顺序排序
    if sort_by_len:
        sorted_index = len_argsort(out_en_sentences)
    #print(sorted_index)
    #sorted_index=[63, 1544, 1917, 2650, 3998, 6240, 6294, 6703, ....
     #前面的索引都是最短句子的索引
      
        out_en_sentences = [out_en_sentences[i] for i in sorted_index]
     #print(out_en_sentences)
     #out_en_sentences=[[2, 475, 4, 3], [2, 1318, 126, 3], [2, 1707, 126, 3], ......
     
        out_cn_sentences = [out_cn_sentences[i] for i in sorted_index]
        
    return out_en_sentences, out_cn_sentences

train_en, train_cn = encode(train_en, train_cn, en_dict, cn_dict)
dev_en, dev_cn = encode(dev_en, dev_cn, en_dict, cn_dict)

In [105]:
k=9654
print(" ".join([inv_cn_dict[i] for i in train_cn[k]])) #通过inv字典获取单词
print(" ".join([inv_en_dict[i] for i in train_en[k]])) 

BOS 今年 的 天氣 一直 異常 。 EOS
BOS the weather has been unusual this year . EOS


### 将语料库分成 batch

In [106]:
def get_minibatches(n, minibatch_size, shuffle=True):
    idx_list = np.arange(0, n, minibatch_size) # [0,15，30，45，60，75，90]
    if shuffle:
        np.random.shuffle(idx_list) #打乱数据 【15，60，30，75，45，0，90】
    minibatches = []
    for idx in idx_list:
        minibatches.append(np.arange(idx, min(idx + minibatch_size, n)))
        #所有batch放在一个大列表里
    return minibatches

In [107]:
def prepare_data(seqs):
    '''
    输入的是
    sequence batch 【【2，4，1】，【1，46，7，23】，【31，4，752，23，74】...】
    
    输出的是 
    长度一致的 sequence batch 【【2，4，1，0，0】，【1，46，7，23，0】，【31，4，752，23，74】...】
    每个句子的长度
    
    '''
#seqs=[[2, 12, 167, 23, 114, 5, 27, 1755, 4, 3], ........
    lengths = [len(seq) for seq in seqs]#每个batch里语句的长度统计出来
    n_samples = len(seqs) #一个batch有多少语句
    max_len = np.max(lengths) #取出最长的的语句长度，后面用这个做padding基准
    x = np.zeros((n_samples, max_len)).astype('int32')
    #先初始化全零矩阵，后面依次赋值
    #print(x.shape) #64*最大句子长度
    
    x_lengths = np.array(lengths).astype("int32")
    #print(x_lengths) 
    #这里看下面的输入语句发现英文句子长度都一样，中文句子长短不一。
    #说明英文句子是特征，中文句子是标签。


    for idx, seq in enumerate(seqs):
      #取出一个batch的每条语句和对应的索引
        x[idx, :lengths[idx]] = seq
        #每条语句按行赋值给x，x会有一些零值没有被赋值。
        
    return x, x_lengths #x_mask

def gen_examples(en_sentences, cn_sentences, batch_size):
    minibatches = get_minibatches(len(en_sentences), batch_size)
    all_ex = []
    for minibatch in minibatches:
        mb_en_sentences = [en_sentences[t] for t in minibatch]
        #按打乱的batch序号分数据，打乱只是batch打乱，一个batach里面的语句还是顺序的。
        #print(mb_en_sentences)
        
        mb_cn_sentences = [cn_sentences[t] for t in minibatch]
        mb_x, mb_x_len = prepare_data(mb_en_sentences)
        #返回的维度为：mb_x=(64 * 最大句子长度）,mb_x_len=最大句子长度
        mb_y, mb_y_len = prepare_data(mb_cn_sentences)
        
        all_ex.append((mb_x, mb_x_len, mb_y, mb_y_len))
        #这里把所有batch数据集合到一起。
        #依次为英文句子，英文长度，中文句子翻译，中文句子长度，这四个放在一个列表中
        #一个列表为一个batch的数据，所有batch组成一个大列表数据

        
    return all_ex


In [108]:

batch_size = 64
train_data = gen_examples(train_en, train_cn, batch_size)
random.shuffle(train_data)
dev_data = gen_examples(dev_en, dev_cn, batch_size)

## 没有 Attention 的 seq2seq 模型

![](https://img-blog.csdnimg.cn/20191127162941278.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzI2NzYxOTQ1,size_16,color_FFFFFF,t_70)
### encoder

In [109]:
class PlainEncoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        #以英文为例，vocab_size=5493, hidden_size=100, dropout=0.2
        super(PlainEncoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        #这里的hidden_size为embedding_dim：一个单词的维度 
        #torch.nn.Embedding(num_embeddings, embedding_dim, .....)
        #这里的hidden_size = 100
        
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)      
        #第一个参数为input_size ：输入特征数量
        #第二个参数为hidden_size ：隐藏层特征数量

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths): 
        #x是输入的batch的所有单词，lengths：batch里每个句子的长度
        #因为需要把最后一个hidden state取出来，需要知道长度，因为句子长度不一样
        ##print(x.shape,lengths),x.sahpe = torch.Size([64, 10])
        # lengths= =tensor([10, 10, 10, ..... 10, 10, 10])
        ''' 把句子按照长度排序 '''
        sorted_len, sorted_idx = lengths.sort(0, descending=True)
        #按照长度排序，descending=True长的在前。
        #返回两个参数，句子长度和未排序前的索引
        # sorted_idx=tensor([41, 40, 46, 45,...... 19, 18, 63])
        # sorted_len=tensor([10, 10, 10, ..... 10, 10, 10])
        
        x_sorted = x[sorted_idx.long()] #句子用新的idx，按长度排好序了
        
        embedded = self.dropout(self.embed(x_sorted))
        #print(embedded.shape)=torch.Size([64, 10, 100])
        #tensor([[[-0.6312, -0.9863, -0.3123,  ..., -0.7384,  0.9230, -0.4311],....
        
        '''
        因为句子的长度不一样，我们获取的最后的 hidden state 位置不一定是 rnn 的最后一个 hidden state 
        直接 将 embedded 进行 rnn 训练的话 默认是 返回的最后一个 hidden state
        '''
        ''' 去掉数据中的补 0 ，因为 function 要求数据要根据长度排序 进入处理 所以上面进行了数据的排序处理'''
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(), batch_first=True)
        #这个函数就是用来处理不同长度的句子的，https: // www.cnblogs.com / sbj123456789 / p / 9834018. html

        packed_out, hid = self.rnn(packed_embedded)
        #hid.shape = torch.Size([1, 64, 100])
        ''' 将补 0 加回来'''
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        #out.shape = torch.Size([64, 10, 100]),
        
        ''' 恢复 batch 数据的原顺序 '''
        _, original_idx = sorted_idx.sort(0, descending=False)
        out = out[original_idx.long()].contiguous()
        hid = hid[:, original_idx.long()].contiguous()
        ''' 
        因为之前的处理 数据在内存中可能不连续了，这里使用 contiguous() 来处理一下数据
        contiguous() 就是将不连续的内存单元连续在一起
        '''
        #out.shape = torch.Size([64, 10, 100])
        #hid.shape = torch.Size([1, 64, 100])
        # 因为 hid 的【num_layers * num_direction, batch_size, hidden_size】
        return out, hid[[-1]] #有时候num_layers层数多，或者是双向的 需要取出最后一层 

### decoder

In [110]:
class PlainDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        super(PlainDecoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, y, y_lengths, hid):
        #print(y.shape)=torch.Size([64, 12])
        #print(hid.shape)=torch.Size([1, 64, 100])
        #中文的y和y_lengths
        ''' 排序 '''
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)
        y_sorted = y[sorted_idx.long()]
        hid = hid[:, sorted_idx.long()] #隐藏层也要排序

        y_sorted = self.dropout(self.embed(y_sorted))
        
        ''' 因为encode的hidden state 传入每一层的输入所以这里要对输入做一个 concatnate 连接处理'''
        #y_sorted = torch.cat([y_sorted,hid.ensqueeze(0).expand_as(y_sorted)],2) 
        #ensqueeze(0) 因为hidden state 只有一个要对每一个输入都添加所以这里对第0维进行扩展
        #expand_as(y_sorted)扩展的维度就是 y_sorted 的维度
        
        # batch_size, output_length, embed_size
        ''' pack 处理'''
        packed_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        ''' rnn 默认不传 hidden state 时 是0，这里我们需要把上encode的最后一层的 hidden state 传过来'''
        out, newhid = self.rnn(packed_seq, hid) #加上隐藏层
        #print(hid.shape)=torch.Size([1, 64, 100])
        
        ''' 恢复 pad'''
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        
        ''' 恢复排序'''
        _, original_idx = sorted_idx.sort(0, descending=False)
        
        output_seq = unpacked[original_idx.long()].contiguous()#print(output_seq.shape)=torch.Size([64, 12, 100])
        newhid = newhid[:, original_idx.long()].contiguous()#print(hid.shape)=torch.Size([1, 64, 100])
        ''' 对输出进行拼接'''
        #output_seq = torch.cat([output_seq,hid.ensqueeze(0).expand_as(y_sorted)],2) 
        ''' 通过 FC 计算结果'''
        output = F.log_softmax(self.out(output_seq), -1)#print(output.shape)=torch.Size([64, 12, 3195])
        
        return output, newhid

### PlainSeq2Seq

In [111]:
class PlainSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        #encoder是上面PlainEncoder的实例
        #decoder是上面PlainDecoder的实例
        super(PlainSeq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
       
    #把两个模型串起来 
    def forward(self, x, x_lengths, y, y_lengths):
        encoder_out, hid = self.encoder(x, x_lengths)
        #self.encoder(x, x_lengths)调用PlainEncoder里面forward的方法
        #返回forward的out和hid
        
        output, hid = self.decoder(y=y,y_lengths=y_lengths,hid=hid)
        #self.dencoder()调用PlainDecoder里面forward的方法
        
        return output, None

    def translate(self, x, x_lengths, y, max_length=10):
        #x是一个句子，用数值表示
        #y是句子的长度
        #y是“bos”的数值索引=2
        
        encoder_out, hid = self.encoder(x, x_lengths)
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):
            output, hid = self.decoder(y=y,
                    y_lengths=torch.ones(batch_size).long().to(y.device),
                    hid=hid) 
            
#刚开始循环bos作为模型的首个输入单词，后续更新y，下个预测单词的输入是上个输出单词
            y = output.max(2)[1].view(batch_size, 1)
            preds.append(y)
            
        return torch.cat(preds, 1), None

### LanguageModelCriterion

In [112]:
# masked cross entropy loss
class LanguageModelCriterion(nn.Module):
    def __init__(self):
        super(LanguageModelCriterion, self).__init__()

    def forward(self, input, target, mask):
        #target=tensor([[5,108,8,4,3,0,0,0,0,0,0,0],....
        #  mask=tensor([[1,1 ,1,1,1,0,0,0,0,0,0,0],.....
        #print(input.shape,target.shape,mask.shape)
        #torch.Size([64, 12, 3195]) torch.Size([64, 12]) torch.Size([64, 12])
        
        # input: (batch_size * seq_len) * vocab_size
        input = input.contiguous().view(-1, input.size(2))
        
        # target: batch_size * 1=768*1
        target = target.contiguous().view(-1, 1)
        mask = mask.contiguous().view(-1, 1)
        #print(-input.gather(1, target))
        output = -input.gather(1, target) * mask
        #这里算得就是交叉熵损失，前面已经算了F.log_softmax
        #.gather的作用https://blog.csdn.net/edogawachia/article/details/80515038
        #output.shape=torch.Size([768, 1])
        #mask作用是把padding为0的地方重置为零，因为input.gather时，为0的地方不是零了
        
        output = torch.sum(output) / torch.sum(mask)
        #均值损失

        return output

### train

In [113]:
def train(model, data, num_epochs=2):
    for epoch in range(num_epochs):
        model.train()
        total_num_words = total_loss = 0.
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            #（英文batch，英文长度，中文batch，中文长度）
            
            mb_x = torch.from_numpy(mb_x).to(device).long()
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()
            
            #前n-1个单词作为输入，后n-1个单词作为输出，因为输入的前一个单词要预测后一个单词
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long()
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long()
            #
            mb_y_len = torch.from_numpy(mb_y_len-1).to(device).long()
            #输入输出的长度都减一。
            
            mb_y_len[mb_y_len<=0] = 1
            
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            #返回的是类PlainSeq2Seq里forward函数的两个返回值
            
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
            #mb_out_mask=tensor([[1, 1, 1,  ..., 0, 0, 0],[1, 1, 1,  ..., 0, 0, 0],
            #mb_out_mask.shape= (64*19),这句代码咱不懂，这个mask就是padding的位置设置为0，其他设置为1
            #mb_out_mask就是LanguageModelCriterion的传入参数mask。

            mb_out_mask = mb_out_mask.float()
            
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            
            num_words = torch.sum(mb_y_len).item()
            #一个batch里多少个单词
            
            total_loss += loss.item() * num_words
            #总损失，loss计算的是均值损失，每个单词都是都有损失，所以乘以单词数
            
            total_num_words += num_words
            #总单词数
            
            # 更新模型
            optimizer.zero_grad()
            loss.backward()
            ''' RNN 防止梯度爆炸 要进行 clip grad'''
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
            #为了防止梯度过大，设置梯度的阈值
            
            optimizer.step()
            
            if it % 100 == 0:
                print("Epoch", epoch, "iteration", it, "loss", loss.item())

                
        print("Epoch", epoch, "Training loss", total_loss/total_num_words)
        if epoch % 5 == 0:
            evaluate(model, dev_data) #评估模型

### evaluate

In [114]:
def evaluate(model, data):
    model.eval()
    total_num_words = total_loss = 0.
    with torch.no_grad():#不需要更新模型，不需要梯度
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long()
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long()
            mb_y_len = torch.from_numpy(mb_y_len-1).to(device).long()
            mb_y_len[mb_y_len<=0] = 1

            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)

            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
            mb_out_mask = mb_out_mask.float()

            loss = loss_fn(mb_pred, mb_output, mb_out_mask)

            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
    print("Evaluation loss", total_loss/total_num_words)


### 开始训练

In [115]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
hidden_size = 100

#传入中文和英文参数
encoder = PlainEncoder(vocab_size=en_total_words,
                      hidden_size=hidden_size,
                      dropout=dropout)
decoder = PlainDecoder(vocab_size=cn_total_words,
                      hidden_size=hidden_size,
                      dropout=dropout)
model = PlainSeq2Seq(encoder, decoder)

model = model.to(device)
loss_fn = LanguageModelCriterion().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [116]:
train(model, train_data, num_epochs=2)

Epoch 0 iteration 0 loss 9.347283363342285
Epoch 0 iteration 100 loss 4.824400424957275
Epoch 0 iteration 200 loss 5.507288932800293
Epoch 0 Training loss 5.870748127984871
Evaluation loss 5.188047265914972
Epoch 1 iteration 0 loss 5.206871032714844
Epoch 1 iteration 100 loss 4.175070762634277
Epoch 1 iteration 200 loss 5.044918060302734
Epoch 1 Training loss 4.945426733946697


### 翻译测试

In [117]:
#翻译个句子看看结果咋样
def translate_dev(i):
    #随便取出句子
    en_sent = " ".join([inv_en_dict[w] for w in dev_en[i]])
    print(en_sent)
    cn_sent = " ".join([inv_cn_dict[w] for w in dev_cn[i]])
    print("".join(cn_sent))

    mb_x = torch.from_numpy(np.array(dev_en[i]).reshape(1, -1)).long().to(device)
    #把句子升维，并转换成tensor
    
    mb_x_len = torch.from_numpy(np.array([len(dev_en[i])])).long().to(device)
    #取出句子长度，并转换成tensor
    
    bos = torch.Tensor([[cn_dict["BOS"]]]).long().to(device)
    #bos=tensor([[2]])

    translation, attn = model.translate(mb_x, mb_x_len, bos)
    #这里传入bos作为首个单词的输入
    #translation=tensor([[ 8,  6, 11, 25, 22, 57, 10,  5,  6,  4]])
    
    translation = [inv_cn_dict[i] for i in translation.data.cpu().numpy().reshape(-1)]
    trans = []
    for word in translation:
        if word != "EOS": # 把数值变成单词形式
            trans.append(word) #
        else:
            break
    print("".join(trans))

for i in range(100,120):
    translate_dev(i)
    print()

BOS you have nice skin . EOS
BOS 你 的 皮膚 真好 。 EOS
你的人。

BOS you 're UNK correct . EOS
BOS 你 UNK 正确 。 EOS
你的人。

BOS everyone admired his courage . EOS
BOS 每個 人 都 佩服 他 的 勇氣 。 EOS
他是他的。

BOS what time is it ? EOS
BOS 几点 了 ？ EOS
他是什么？

BOS i 'm free tonight . EOS
BOS 我 今晚 有空 。 EOS
我不知道。

BOS here is your book . EOS
BOS 這是 你 的 書 。 EOS
你的。

BOS they are at lunch . EOS
BOS 他们 在 吃 午饭 。 EOS
我在这里。

BOS this chair is UNK . EOS
BOS 這把 椅子 UNK 。 EOS
他是我的。

BOS it 's pretty heavy . EOS
BOS 它 UNK 。 EOS
他是我的。

BOS many attended his funeral . EOS
BOS 很多 人 都 参加 了 他 的 UNK 。 EOS
他是他的。

BOS training will be provided . EOS
BOS 会 有 训练 。 EOS
她的人。

BOS someone is watching you . EOS
BOS 有人 在 看 著 你 。 EOS
你的。

BOS i slapped his face . EOS
BOS 我 摑 了 他 的 臉 。 EOS
我不。

BOS i like UNK music . EOS
BOS 我 喜歡 流行 音樂 。 EOS
我不知道。

BOS tom had no children . EOS
BOS Tom 沒有 孩子 。 EOS
汤姆是我的。

BOS please lock the door . EOS
BOS 請 把 UNK 上 。 EOS
我的。

BOS tom has calmed down . EOS
BOS 汤姆 冷静下来 了 。 EOS
汤姆是我的。

BOS please speak more loudl